<a href="https://colab.research.google.com/github/Tiger-A/GoogleCollab/blob/main/ml_for_texts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.6 MB/s eta 0:00:00


In [2]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.6 MB/s eta 0:00:00


In [58]:
import pandas as pd
import numpy as np
import re
from google.colab import drive


import torch
# import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score)
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    train_test_split
    )



import transformers
from transformers import BertModel, BertTokenizer
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer, f1_score

In [6]:

try:
    '''путь loc'''
    drive.mount('/content/drive')
    df_tweets = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ds_yandex/db/toxic_comments.csv')

except:
    '''путь для ревьюера'''
    df_tweets = pd.read_csv('/datasets/toxic_comments.csv')

Mounted at /content/drive


In [7]:
#  Для тестов возьмем 500
df_tweets = df_tweets.sample(500).reset_index(drop=True)

In [ ]:
# todo исследование баланса классов

In [8]:
def clear_text(text):
    # text = re.sub(r'[^а-яА-ЯёЁ ]', ' ', text)
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    text = " ".join(text.split())
#    print(text)

    return text

df_tweets['text'] = df_tweets['text'].apply(clear_text)



In [9]:
df_tweets['text']

0                        cosmetic i e flip vs slider etc
1      Discussion boards are almost the lowest form o...
2      Where can I read the vandalism site Please tel...
3      MrBigB I m not sure how many reverts I ve done...
4      Please google image flag of northern ireland n...
                             ...                        
495    More CG generated pictures Ag eyban Since you ...
496    Please do not mistake my playful ribs as warni...
497                   and redirect the other names to it
498    UTC On a similar note do we have a source for ...
499    Check than American Kennel Club breed standard...
Name: text, Length: 500, dtype: object

In [10]:
features = df_tweets.drop(['toxic'], axis=1)
target = df_tweets['toxic']

In [11]:
# Загрузка предобученного токенизатора
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

# Загрузка предобученной модели
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
# построчно Преобразуем текст в номера токенов из словаря методом encode() (англ. «закодировать»):
tokenized = df_tweets['text'].apply(
    lambda x: tokenizer.encode(x[:512], add_special_tokens=True))
#  Вычислим макс длину токена
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
# Применим метод padding (англ. «отступ»), чтобы после токенизации длины исходных текстов в корпусе были равными. Только при таком условии будет работать модель BERT.
# Пусть стандартной длиной вектора n будет длина наибольшего во всём датасете вектора. Остальные векторы дополним нулями:
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
# Теперь поясним модели, что нули не несут значимой информации. Это нужно для компоненты модели, которая называется «внимание» (англ. attention).
# Отбросим эти токены и «создадим маску» для действительно важных токенов, то есть укажем нулевые и не нулевые значения:
attention_mask = np.where(padded != 0, 1, 0)

In [13]:
attention_mask

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [ ]:
attention_mask.shape


(500, 258)

In [14]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/5 [00:00<?, ?it/s]

In [15]:
features = np.concatenate(embeddings)
# обучите и протестируйте модель
# < напишите код здесь >


X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.5, random_state=12345
)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(250, 768)
(250, 768)
(250,)
(250,)


## Обучение

In [16]:
# LR
# model = LogisticRegression()
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
pred = model.predict(X_test)

In [17]:
f1 = f1_score(y_test, pred)

In [18]:
print('F1 Score "LR":', f1)

F1 Score "LR": 0.41379310344827586


In [61]:
%%time

f1_scorer = make_scorer(f1_score)

#  пайплайн
pipeline = Pipeline([

    ('regressor', CatBoostRegressor())
])

# сетки параметров
param_grid = {
    'regressor__learning_rate': [0.1, 0.001],  # Параметры модели CatBoostRegressor
    'regressor__depth': [3, 7]
}

grid_search_catboost = GridSearchCV(pipeline, param_grid=param_grid, cv=5, scoring=f1_scorer)

# пееберем параметры
grid_search_catboost.fit(X_train, y_train)

gs_catboost_best_score = grid_search_catboost.best_score_
print("Лучшие параметры:", grid_search_catboost.best_params_)
print("Значение F1 Score:", gs_catboost_best_score)


0:	learn: 0.3214952	total: 41.1ms	remaining: 41.1s
1:	learn: 0.3178622	total: 75ms	remaining: 37.4s
2:	learn: 0.3151235	total: 109ms	remaining: 36.3s
3:	learn: 0.3101807	total: 142ms	remaining: 35.4s
4:	learn: 0.3081251	total: 175ms	remaining: 34.8s
5:	learn: 0.3030489	total: 208ms	remaining: 34.5s
6:	learn: 0.2982765	total: 243ms	remaining: 34.4s
7:	learn: 0.2959932	total: 277ms	remaining: 34.4s
8:	learn: 0.2925041	total: 314ms	remaining: 34.5s
9:	learn: 0.2873244	total: 351ms	remaining: 34.7s
10:	learn: 0.2832111	total: 384ms	remaining: 34.5s
11:	learn: 0.2792953	total: 417ms	remaining: 34.3s
12:	learn: 0.2766812	total: 453ms	remaining: 34.4s
13:	learn: 0.2723988	total: 486ms	remaining: 34.2s
14:	learn: 0.2705904	total: 520ms	remaining: 34.1s
15:	learn: 0.2640823	total: 553ms	remaining: 34s
16:	learn: 0.2624196	total: 586ms	remaining: 33.9s
17:	learn: 0.2590561	total: 619ms	remaining: 33.8s
18:	learn: 0.2571882	total: 652ms	remaining: 33.6s
19:	learn: 0.2557882	total: 690ms	remaining

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1287, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(
  File "/usr/local/l

0:	learn: 0.3224116	total: 57.4ms	remaining: 57.4s
1:	learn: 0.3185643	total: 91ms	remaining: 45.4s
2:	learn: 0.3152773	total: 132ms	remaining: 43.7s
3:	learn: 0.3112574	total: 170ms	remaining: 42.2s
4:	learn: 0.3077877	total: 201ms	remaining: 40.1s
5:	learn: 0.3044140	total: 234ms	remaining: 38.7s
6:	learn: 0.3026321	total: 270ms	remaining: 38.3s
7:	learn: 0.3001601	total: 304ms	remaining: 37.7s
8:	learn: 0.2934943	total: 336ms	remaining: 36.9s
9:	learn: 0.2900868	total: 368ms	remaining: 36.4s
10:	learn: 0.2868946	total: 400ms	remaining: 35.9s
11:	learn: 0.2841309	total: 433ms	remaining: 35.6s
12:	learn: 0.2813910	total: 466ms	remaining: 35.4s
13:	learn: 0.2770515	total: 502ms	remaining: 35.4s
14:	learn: 0.2743413	total: 535ms	remaining: 35.1s
15:	learn: 0.2699292	total: 581ms	remaining: 35.7s
16:	learn: 0.2685806	total: 615ms	remaining: 35.5s
17:	learn: 0.2652579	total: 647ms	remaining: 35.3s
18:	learn: 0.2617785	total: 681ms	remaining: 35.1s
19:	learn: 0.2601042	total: 718ms	remaini

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1287, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(
  File "/usr/local/l

0:	learn: 0.3027832	total: 39.2ms	remaining: 39.1s
1:	learn: 0.2980123	total: 71.2ms	remaining: 35.5s
2:	learn: 0.2944503	total: 114ms	remaining: 37.8s
3:	learn: 0.2908979	total: 147ms	remaining: 36.7s
4:	learn: 0.2838946	total: 179ms	remaining: 35.7s
5:	learn: 0.2809895	total: 215ms	remaining: 35.7s
6:	learn: 0.2781219	total: 254ms	remaining: 36.1s
7:	learn: 0.2727717	total: 288ms	remaining: 35.7s
8:	learn: 0.2665073	total: 320ms	remaining: 35.2s
9:	learn: 0.2605594	total: 352ms	remaining: 34.9s
10:	learn: 0.2551385	total: 385ms	remaining: 34.6s
11:	learn: 0.2510809	total: 417ms	remaining: 34.4s
12:	learn: 0.2488324	total: 451ms	remaining: 34.2s
13:	learn: 0.2460632	total: 487ms	remaining: 34.3s
14:	learn: 0.2416502	total: 519ms	remaining: 34.1s
15:	learn: 0.2355827	total: 552ms	remaining: 33.9s
16:	learn: 0.2336409	total: 586ms	remaining: 33.9s
17:	learn: 0.2309459	total: 618ms	remaining: 33.7s
18:	learn: 0.2281191	total: 656ms	remaining: 33.9s
19:	learn: 0.2251795	total: 693ms	remai

...

0:	learn: 0.3306456	total: 702ms	remaining: 11m 40s
1:	learn: 0.3305889	total: 1.23s	remaining: 10m 13s
2:	learn: 0.3305037	total: 1.74s	remaining: 9m 39s
3:	learn: 0.3304353	total: 2.21s	remaining: 9m 11s
4:	learn: 0.3303686	total: 2.5s	remaining: 8m 18s
5:	learn: 0.3302923	total: 2.78s	remaining: 7m 40s
6:	learn: 0.3302400	total: 3.06s	remaining: 7m 14s
7:	learn: 0.3301618	total: 3.35s	remaining: 6m 55s
8:	learn: 0.3301069	total: 3.62s	remaining: 6m 38s
9:	learn: 0.3300246	total: 3.89s	remaining: 6m 25s
10:	learn: 0.3299743	total: 4.18s	remaining: 6m 15s
11:	learn: 0.3299007	total: 4.45s	remaining: 6m 6s
12:	learn: 0.3298078	total: 4.73s	remaining: 5m 59s
13:	learn: 0.3297458	total: 5.01s	remaining: 5m 52s
14:	learn: 0.3296933	total: 5.29s	remaining: 5m 47s
15:	learn: 0.3296116	total: 5.57s	remaining: 5m 42s
16:	learn: 0.3295525	total: 5.84s	remaining: 5m 37s
17:	learn: 0.3294808	total: 6.12s	remaining: 5m 34s
18:	learn: 0.3293946	total: 6.4s	remaining: 5m 30s
19:	learn: 0.3293430	to

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1287, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(
  File "/usr/local/l

0:	learn: 0.3127869	total: 379ms	remaining: 6m 18s
1:	learn: 0.3127310	total: 666ms	remaining: 5m 32s
2:	learn: 0.3126530	total: 941ms	remaining: 5m 12s
3:	learn: 0.3125710	total: 1.22s	remaining: 5m 2s
4:	learn: 0.3124976	total: 1.51s	remaining: 4m 59s
5:	learn: 0.3124133	total: 1.78s	remaining: 4m 54s
6:	learn: 0.3123261	total: 2.09s	remaining: 4m 55s
7:	learn: 0.3122661	total: 2.56s	remaining: 5m 17s
8:	learn: 0.3121977	total: 3.07s	remaining: 5m 37s
9:	learn: 0.3121273	total: 3.59s	remaining: 5m 55s
10:	learn: 0.3120449	total: 4.07s	remaining: 6m 5s
11:	learn: 0.3119604	total: 4.59s	remaining: 6m 18s
12:	learn: 0.3118864	total: 5.13s	remaining: 6m 29s
13:	learn: 0.3118113	total: 5.63s	remaining: 6m 36s
14:	learn: 0.3117256	total: 6.19s	remaining: 6m 46s
15:	learn: 0.3116797	total: 6.76s	remaining: 6m 55s
16:	learn: 0.3116035	total: 7.29s	remaining: 7m 1s
17:	learn: 0.3115288	total: 7.78s	remaining: 7m 4s
18:	learn: 0.3114650	total: 8.3s	remaining: 7m 8s
19:	learn: 0.3113869	total: 

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1287, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(
  File "/usr/local/l

0:	learn: 0.3171966	total: 54ms	remaining: 53.9s
1:	learn: 0.3144787	total: 103ms	remaining: 51.5s
2:	learn: 0.3125360	total: 151ms	remaining: 50.1s
3:	learn: 0.3098084	total: 192ms	remaining: 47.7s
4:	learn: 0.3052245	total: 234ms	remaining: 46.6s
5:	learn: 0.2988896	total: 296ms	remaining: 49s
6:	learn: 0.2947856	total: 337ms	remaining: 47.8s
7:	learn: 0.2931497	total: 378ms	remaining: 46.8s
8:	learn: 0.2919241	total: 420ms	remaining: 46.3s
9:	learn: 0.2881187	total: 461ms	remaining: 45.7s
10:	learn: 0.2839912	total: 514ms	remaining: 46.2s
11:	learn: 0.2807392	total: 560ms	remaining: 46.1s
12:	learn: 0.2787606	total: 603ms	remaining: 45.8s
13:	learn: 0.2744866	total: 644ms	remaining: 45.4s
14:	learn: 0.2709415	total: 685ms	remaining: 45s
15:	learn: 0.2668180	total: 736ms	remaining: 45.2s
16:	learn: 0.2638724	total: 789ms	remaining: 45.6s
17:	learn: 0.2631121	total: 835ms	remaining: 45.5s
18:	learn: 0.2607295	total: 877ms	remaining: 45.3s
19:	learn: 0.2556733	total: 921ms	remaining: 4


 Лучшие параметры: {'regressor__depth': 3, 'regressor__learning_rate': 0.1}
Значение F1 Score: nan
CPU times: user 1h 36min 46s, sys: 28.4 s, total: 1h 37min 14s
Wall time: 1h 6min 48s

In [62]:
# Получаем лучшие параметры и модель с лучшими параметрами
best_model_catboost = grid_search_catboost.best_estimator_

In [63]:
!pip install h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [64]:
X_train = pd.DataFrame(X_train)
#  y_train

In [65]:


train = pd.concat([X_train, y_train], axis=1)

In [38]:
# train['toxic'] = train['toxic'].astype('category')

In [66]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 372 entries, 0 to 482
Columns: 769 entries, 0 to toxic
dtypes: float32(768), float64(1)
memory usage: 1.1 MB


In [67]:
import h2o
from h2o.automl import H2OAutoML

In [68]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,3 hours 37 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,1 month and 23 days
H2O_cluster_name:,H2O_from_python_unknownUser_wrz7vb
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.038 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [69]:
h2o_train = h2o.H2OFrame(train)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [70]:
# Преобразование столбца с ответами в категориальный тип данных
h2o_train['toxic'] = h2o_train['toxic'].asfactor()

In [71]:
y = 'toxic'

In [72]:
aml = H2OAutoML(max_runtime_secs=360)  # Укажите максимальное время работы в секундах
aml.train(y=y, training_frame=h2o_train)

AutoML progress: |
10:26:52.89: XGBoost_1_AutoML_3_20230621_102651 [XGBoost def_2] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_1_AutoML_3_20230621_102651_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


████████████
10:27:49.160: XGBoost_2_AutoML_3_20230621_102651 [XGBoost def_1] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_2_AutoML_3_20230621_102651_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


██████████
10:28:49.992: XGBoost_3_AutoML_3_20230621_102651 [XGBoost def_3] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_3_AutoML_3_20230621_102651_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by 

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_grid_1_AutoML_3_20230621_102651_model_3


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    23                 23                          4849                   4            10           8.73913       6             16            12.2174

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.04978879737774794
RMSE: 0.22313403455714223
LogLoss: 0.1947427277329161
Mean Per-Class Error: 0.20689655172413793
AUC: 0.8932750819160555
AUCPR: 0.7297716520984399
Gini: 0.7865501638321111

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5676833427729308
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      221  0    0        (0.0/221.0)
1      12   17   0.4138   (12.0/29.0)
Total  233  17   0.048    (12.0/250.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.567683     0.73913   16
max f2                       0.567683     0.639098  16
max f0point5                 0.567683     0.876289  16
max accuracy                 0.567683     0.952     16
max precision                0.816713     1         0
max recall                   0.0947415    1         19
max specificity              0.816713     1         0
max absolute_mcc             0.567683     0.745665  16
max min_per_class_accuracy   0.567683     0.586207  16
max mean_per_class_accuracy  0.567683     0.793103  16
max tns                      0.816713     221       0
max fns                      0.816713     28        0
max fps                      0.0126926    221       128
max tps                      0.0947415    29        19
max tnr                      0.816713     1         0
max fnr                      0.816713     0.965517  0
max fpr                      0.0126926    1         128
max tpr                      0.0947415    1         19

Gains/Lift Table: Avg response rate: 11.60 %, avg score: 10.89 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.016                       0.756421           8.62069   8.62069            1                0.794468   1                           0.794468            0.137931        0.137931                   762.069   762.069            0.137931
2        0.032                       0.704303           8.62069   8.62069            1                0.723882   1                           0.759175            0.137931        0.275862                   762.069   762.069            0.275862
3        0.048                       0.689463           8.62069   8.62069            1                0.6942     1                           0.737517            0.137931        0.413793                   762.069   762.069            0.413793
4        0.06                        0.648327           8.62069   8.62069            1                0.683212   1                           0.726656            0.103448        0.517241                   762.069   762.069            0.517241
5        0.076                       0.0958375          4.31034   7.71325            0.5              0.351799   0.894737                    0.647739            0.06

In [57]:
# # Можно посмотреть на результаты других моделей - AutoML Leaderboard
# lb = aml.leaderboard
# lb.head(rows=lb.nrows)  # Выведем все строки

In [73]:
# Получим лучшую модель
best_model_h2o = aml.leader

In [79]:
X_test = pd.DataFrame(X_test)
#  y_train

In [80]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Columns: 768 entries, 0 to 767
dtypes: float32(768)
memory usage: 750.1 KB


In [84]:
test_h2o = pd.concat([X_test, y_test], axis=1)

In [85]:
h2o_test = h2o.H2OFrame(test_h2o)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [91]:
y_pred = best_model_h2o.predict(h2o_test)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [92]:
y_pred

predict,p0,p1
0,0.926915,0.0730852
0,0.975933,0.0240674
0,0.976079,0.0239213
0,0.932951,0.0670488
0,0.964138,0.0358618
0,0.945944,0.0540564
0,0.979787,0.0202131
0,0.962975,0.0370248
0,0.987414,0.0125857
0,0.668026,0.331974


In [87]:
y_pred_np = np.array(y_pred.as_data_frame()['predict'])

In [90]:
y_pred_np

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [89]:
f1 = f1_score(y_test, y_pred_np)

ValueError: ignored

## Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны